In [27]:


class TermExtractor:
    def __init__(self, extraction_methods):
        self.extraction_methods = extraction_methods
        self.extractors = self.initialize_keyword_extractors()

    def __call__(self, text):
        self.extract_terms(text)
        
    def initialize_keyword_extractors(self):
        keyword_extractors = {}

        if 'rake' in self.extraction_methods:
            keyword_extractors["rake"] = "OBJETO YAKE EXTRACTOR" #YakeExtractor()
            
        if 'textrank' in self.extraction_methods:
            keyword_extractors["textrank"] = "OBJETO TEXTRANK EXTRACTOR"
        
        return keyword_extractors

    def extract_terms(self, text):
        print(self.extractors)
        print(text)
    

In [28]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [29]:
extractor("texto de prueba")

{'rake': 'OBJETO YAKE EXTRACTOR', 'textrank': 'OBJETO TEXTRANK EXTRACTOR'}
texto de prueba


In [15]:
extractor.extractors

{'yake': 'OBJETO YAKE EXTRACTOR'}

In [ ]:
extractor_terminos = TermExtractor(sadasdasdsa)

In [ ]:
doc1_terms = extractor_terminos("textoasdasdasdsa")

In [ ]:
doc1_terms.zº

In [31]:
import sys, os, re
general_path = os.getcwd().split("BioTermCategorizer")[0]+"BioTermCategorizer/"
sys.path.append(general_path+'biotermcategorizer/')


In [36]:
from TermExtractor import TermExtractor

In [37]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [39]:
extractor("texto de prueba")

{'rake': 'OBJETO YAKE EXTRACTOR', 'textrank': 'OBJETO TEXTRANK EXTRACTOR'}
texto de prueba


In [52]:
class Extractor:
    def __init__(self):
        pass

    def extract_terms(self, text):
        raise NotImplementedError("extract_terms method must be implemented in subclass")



In [53]:


class RakeExtractor(Extractor):
    def __init__(self, stopwords, language):
        super().__init__()
        self.stopwords = stopwords
        self.language = language

    def extract_terms(self, text):
        # Lógica para la extracción de términos mediante string matching
        terms = []  # Ejemplo de términos encontrados
        return terms


In [54]:
rake_extractor = RakeExtractor(stopwords=["a","b","c"], language = "spanish")

In [55]:
rake_extractor.extract_terms("hola que tal")

NotImplementedError: extract_terms method must be implemented in subclass